In [20]:
import pandas as pd
import importlib
import os
from ast import literal_eval
import numpy as np
import pickle
import gc
from sklearn.model_selection import train_test_split


In [21]:
#df.to_pickle('../../data/song_train_merged_transferkey_pkl0430') 
df = pd.read_pickle('../../data/song_train_merged_transferkey_pkl0430')

# df cleaning

In [22]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,_id,artist,chord,error,rating,tab_href,tab_title,version,song_name,song_detector_chord,song_chord
0,0,0,28,5cada0049fc5af34a460ae05,10cc,"[G:maj, C:maj, G:maj, Eb:maj, G:maj, G:maj, N,...",0,9,https://tabs.ultimate-guitar.com/tab/10cc/art_...,Art For Arts Sake,Ver 1,Art For Arts Sake 10cc,"[{'st': 0, 'et': 0.511, 'ochord': 'N'}, {'st':...","[N, D:maj, F:maj, D:maj, G:maj, G:maj, Db:maj,..."
1,1,1,29,5cada0049fc5af34a460ae07,10cc,"[G:maj, N, C:maj, Bb:maj, N, G:maj, C:maj, A:m...",0,3,https://tabs.ultimate-guitar.com/tab/10cc/baro...,Baron Samedi,Ver 1,Baron Samedi 10cc,"[{'st': 0, 'et': 3.413, 'ochord': 'A:min'}, {'...","[A:min, G:maj, F:maj, C:maj, D:maj, C:maj, Bb:..."
2,2,2,30,5cada0049fc5af34a460ae0b,10cc,"[Bb:maj, Bb:maj, C:maj, C:maj, Bb:maj, Bb:maj,...",0,4,https://tabs.ultimate-guitar.com/tab/10cc/bran...,Brand New Day,Ver 1,Brand New Day 10cc,"[{'st': 0, 'et': 0.58, 'ochord': 'C'}, {'st': ...","[C:maj, F:min, Bb:maj, G:maj, C:maj, F:min, Bb..."
3,3,3,31,5cada0049fc5af34a460ae0d,10cc,"[Bb:maj, Eb:maj, Bb:maj, C:maj, G:maj, C:maj, ...",0,5,https://tabs.ultimate-guitar.com/tab/10cc/donn...,Donna,Ver 1,Donna 10cc,"[{'st': 0, 'et': 5.782, 'ochord': 'A#'}, {'st'...","[Bb:maj, Bb:maj, C:maj, Eb:maj, Bb:maj, C:maj,..."
4,4,4,32,5cada0049fc5af34a460ae0f,10cc,"[C:maj, A:min, G:maj, F:maj, A:min, D:maj, A:m...",0,75,https://tabs.ultimate-guitar.com/tab/10cc/drea...,Dreadlock Holiday,Ver 1,Dreadlock Holiday 10cc,"[{'st': 0, 'et': 1.254, 'ochord': 'N'}, {'st':...","[N, B:maj, Bb:maj, G:maj, C:maj, G:maj, C:maj,..."


把rating欄位轉成float型態，轉完後可能會有一些row變成NAN，把這些row變成0.0

In [23]:

df['rating']= pd.to_numeric(df['rating'], errors='coerce').fillna(0)

清除outlier:任何一首歌曲和弦數目太多或太少都不要，以便之後做padding <br>
用四分位數檢視

In [24]:
def status(x) : 
    return pd.Series([x.count(),x.min(),x.idxmin(),x.quantile(.05),x.quantile(.25),x.median(),
                      x.quantile(.75),x.quantile(.95),x.mean(),x.max(),x.idxmax(),x.mad(),x.var(),
                      x.std(),x.skew(),x.kurt()],index=['總數','最小值','最小值位置','5%分位數','25%分位數',
                    '中位數','75%分位數','95%分位數','均值','最大值','最大值位數','平均絕對偏差','方差','標準差','偏度','峰度'])


In [25]:
print(status(df['song_chord'].str.len()),status(df['chord'].str.len()))


總數        20102.000000
最小值          10.000000
最小值位置      9269.000000
5%分位數        49.000000
25%分位數       75.000000
中位數          95.000000
75%分位數      117.000000
95%分位數      157.000000
均值          104.911452
最大值        3954.000000
最大值位數     16425.000000
平均絕對偏差       33.794768
方差        12884.677796
標準差         113.510695
偏度           15.669242
峰度          303.136858
dtype: float64 總數        20102.000000
最小值           0.000000
最小值位置        87.000000
5%分位數        24.000000
25%分位數       55.000000
中位數          80.000000
75%分位數      106.000000
95%分位數      159.000000
均值           83.809919
最大值         770.000000
最大值位數      1938.000000
平均絕對偏差       32.350549
方差         1910.253356
標準差          43.706445
偏度            1.618664
峰度            9.751067
dtype: float64


發現chord欄位的95%是159，chord欄位的5%是24 <br>
選擇此兩區間以去除outlier

In [26]:
index=(df['song_chord'].str.len()<159)  & (df['song_chord'].str.len()>24) 
df=df[index]
index=(df['chord'].str.len()<159)  & (df['chord'].str.len()>24) 
df=df[index]

In [27]:
print(status(df['song_chord'].str.len()),status(df['chord'].str.len()))


總數        17309.000000
最小值          25.000000
最小值位置       797.000000
5%分位數        50.000000
25%分位數       75.000000
中位數          93.000000
75%分位數      112.000000
95%分位數      140.000000
均值           93.656133
最大值         158.000000
最大值位數      1541.000000
平均絕對偏差       21.821889
方差          716.681379
標準差          26.770905
偏度            0.061189
峰度           -0.479872
dtype: float64 總數        17309.000000
最小值          25.000000
最小值位置       685.000000
5%分位數        34.000000
25%分位數       58.000000
中位數          79.000000
75%分位數      103.000000
95%分位數      136.000000
均值           81.226703
最大值         158.000000
最大值位數      1650.000000
平均絕對偏差       25.164095
方差          934.335014
標準差          30.566894
偏度            0.282784
峰度           -0.611973
dtype: float64


做完df cleaning，存起來之後就直接讀取

In [36]:
#df.to_pickle('../../data/model_df_cleaned') 
df = pd.read_pickle('../../data/model_df_cleaned')

In [37]:
len(df)

17309

In [38]:
print('共:',len(df.groupby('song_name').size()),'首歌')
print('其中有:',sum(df.groupby('song_name').size()>1),'首歌擁有兩張以上的譜')

共: 9970 首歌
其中有: 3517 首歌擁有兩張以上的譜


找出擁有兩張譜以上的歌 <br>
挑前1000首歌出來之後作為模型的test song

In [39]:
#擁有兩張譜以上的歌
#3517筆
has_2ver=df.groupby('song_name').size().loc[lambda x: x>1].keys()


training song:過濾掉前1000首擁有兩張譜以上的歌(3517的前1000) <br>
test song:選擇前1000首擁有兩張譜以上的歌(3517的前1000)

In [31]:

df_remove_test=df[~df.song_name.isin(has_2ver[:1000])]
df_remove_test

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,_id,artist,chord,error,rating,tab_href,tab_title,version,song_name,song_detector_chord,song_chord
0,0,0,28,5cada0049fc5af34a460ae05,10cc,"[G:maj, C:maj, G:maj, Eb:maj, G:maj, G:maj, N,...",0,9.0,https://tabs.ultimate-guitar.com/tab/10cc/art_...,Art For Arts Sake,Ver 1,Art For Arts Sake 10cc,"[{'st': 0, 'et': 0.511, 'ochord': 'N'}, {'st':...","[N, D:maj, F:maj, D:maj, G:maj, G:maj, Db:maj,..."
1,1,1,29,5cada0049fc5af34a460ae07,10cc,"[G:maj, N, C:maj, Bb:maj, N, G:maj, C:maj, A:m...",0,3.0,https://tabs.ultimate-guitar.com/tab/10cc/baro...,Baron Samedi,Ver 1,Baron Samedi 10cc,"[{'st': 0, 'et': 3.413, 'ochord': 'A:min'}, {'...","[A:min, G:maj, F:maj, C:maj, D:maj, C:maj, Bb:..."
2,2,2,30,5cada0049fc5af34a460ae0b,10cc,"[Bb:maj, Bb:maj, C:maj, C:maj, Bb:maj, Bb:maj,...",0,4.0,https://tabs.ultimate-guitar.com/tab/10cc/bran...,Brand New Day,Ver 1,Brand New Day 10cc,"[{'st': 0, 'et': 0.58, 'ochord': 'C'}, {'st': ...","[C:maj, F:min, Bb:maj, G:maj, C:maj, F:min, Bb..."
3,3,3,31,5cada0049fc5af34a460ae0d,10cc,"[Bb:maj, Eb:maj, Bb:maj, C:maj, G:maj, C:maj, ...",0,5.0,https://tabs.ultimate-guitar.com/tab/10cc/donn...,Donna,Ver 1,Donna 10cc,"[{'st': 0, 'et': 5.782, 'ochord': 'A#'}, {'st'...","[Bb:maj, Bb:maj, C:maj, Eb:maj, Bb:maj, C:maj,..."
10,10,10,38,5cada0069fc5af34a460ae1f,10cc,"[G:maj, N, Gb:maj, G:maj, G:maj, G:maj, G:maj,...",0,4.0,https://tabs.ultimate-guitar.com/tab/10cc/good...,Good Morning Judge,Ver 1,Good Morning Judge 10cc,"[{'st': 0, 'et': 1.625, 'ochord': 'N'}, {'st':...","[N, A:maj, C:maj, A:maj, Gb:maj, A:maj, Gb:maj..."
12,12,12,40,5cada0069fc5af34a460ae29,10cc,"[Gb:min, B:maj, Gb:min, B:maj, Gb:min, B:maj, ...",0,222.0,https://tabs.ultimate-guitar.com/tab/10cc/im_n...,Im Not In Love,Ver 1,Im Not In Love 10cc,"[{'st': 0, 'et': 0.627, 'ochord': 'N'}, {'st':...","[N, Db:maj, A:maj, B:maj, C:maj, A:maj, B:maj,..."
13,13,13,41,5cada0069fc5af34a460ae2b,10cc,"[A:maj, B:maj, Ab:maj, A:maj, A:min, Ab:min, A...",0,34.0,https://tabs.ultimate-guitar.com/tab/10cc/im_n...,Im Not In Love,Ver 2,Im Not In Love 10cc,"[{'st': 0, 'et': 0.627, 'ochord': 'N'}, {'st':...","[N, Db:maj, A:maj, B:maj, C:maj, A:maj, B:maj,..."
14,14,14,42,5cada0069fc5af34a460ae2d,10cc,"[A:maj, A:maj, B:maj, B:maj, A:maj, A:maj, B:m...",0,38.0,https://tabs.ultimate-guitar.com/tab/10cc/im_n...,Im Not In Love,Ver 3,Im Not In Love 10cc,"[{'st': 0, 'et': 0.627, 'ochord': 'N'}, {'st':...","[N, Db:maj, A:maj, B:maj, C:maj, A:maj, B:maj,..."
15,15,15,43,5cada0079fc5af34a460ae2f,10cc,"[Ab:min, Db:min, N, B:maj, B:maj, Ab:min, A:ma...",0,41.0,https://tabs.ultimate-guitar.com/tab/10cc/im_n...,Im Not In Love,Ver 4,Im Not In Love 10cc,"[{'st': 0, 'et': 0.627, 'ochord': 'N'}, {'st':...","[N, Db:maj, A:maj, B:maj, C:maj, A:maj, B:maj,..."
16,16,16,46,5cada0079fc5af34a460ae35,10cc,"[F:maj, G:maj, F:maj, G:maj, A:maj, F:maj, D:m...",0,3.0,https://tabs.ultimate-guitar.com/tab/10cc/john...,Johnny Dont Do It,Ver 1,Johnny Dont Do It 10cc,"[{'st': 0, 'et': 2.345, 'ochord': 'A#'}, {'st'...","[Bb:maj, C:maj, D:maj, Bb:maj, G:maj, C:maj, B..."


# QA架構:

每組song_name取song_chord作為Q;guitar_tab_chord of Max rating作為good answer  <br>
Training_data:11360首只有一張譜的歌+3145首有兩張以上譜的歌 <br>
Testing_data:1000首擁有兩張以上譜的歌

準備資料:
從df2整理出Q與GoodA


In [41]:
#df2:Good Answer的df，每首歌曲最高rating的譜
idx = df_remove_test.groupby(['song_name'])['rating'].transform(max) == df_remove_test['rating']
df2=df_remove_test[idx].copy()
df2

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,_id,artist,chord,error,rating,tab_href,tab_title,version,song_name,song_detector_chord,song_chord
0,0,0,28,5cada0049fc5af34a460ae05,10cc,"[G:maj, C:maj, G:maj, Eb:maj, G:maj, G:maj, N,...",0,9.0,https://tabs.ultimate-guitar.com/tab/10cc/art_...,Art For Arts Sake,Ver 1,Art For Arts Sake 10cc,"[{'st': 0, 'et': 0.511, 'ochord': 'N'}, {'st':...","[N, D:maj, F:maj, D:maj, G:maj, G:maj, Db:maj,..."
1,1,1,29,5cada0049fc5af34a460ae07,10cc,"[G:maj, N, C:maj, Bb:maj, N, G:maj, C:maj, A:m...",0,3.0,https://tabs.ultimate-guitar.com/tab/10cc/baro...,Baron Samedi,Ver 1,Baron Samedi 10cc,"[{'st': 0, 'et': 3.413, 'ochord': 'A:min'}, {'...","[A:min, G:maj, F:maj, C:maj, D:maj, C:maj, Bb:..."
2,2,2,30,5cada0049fc5af34a460ae0b,10cc,"[Bb:maj, Bb:maj, C:maj, C:maj, Bb:maj, Bb:maj,...",0,4.0,https://tabs.ultimate-guitar.com/tab/10cc/bran...,Brand New Day,Ver 1,Brand New Day 10cc,"[{'st': 0, 'et': 0.58, 'ochord': 'C'}, {'st': ...","[C:maj, F:min, Bb:maj, G:maj, C:maj, F:min, Bb..."
3,3,3,31,5cada0049fc5af34a460ae0d,10cc,"[Bb:maj, Eb:maj, Bb:maj, C:maj, G:maj, C:maj, ...",0,5.0,https://tabs.ultimate-guitar.com/tab/10cc/donn...,Donna,Ver 1,Donna 10cc,"[{'st': 0, 'et': 5.782, 'ochord': 'A#'}, {'st'...","[Bb:maj, Bb:maj, C:maj, Eb:maj, Bb:maj, C:maj,..."
10,10,10,38,5cada0069fc5af34a460ae1f,10cc,"[G:maj, N, Gb:maj, G:maj, G:maj, G:maj, G:maj,...",0,4.0,https://tabs.ultimate-guitar.com/tab/10cc/good...,Good Morning Judge,Ver 1,Good Morning Judge 10cc,"[{'st': 0, 'et': 1.625, 'ochord': 'N'}, {'st':...","[N, A:maj, C:maj, A:maj, Gb:maj, A:maj, Gb:maj..."
12,12,12,40,5cada0069fc5af34a460ae29,10cc,"[Gb:min, B:maj, Gb:min, B:maj, Gb:min, B:maj, ...",0,222.0,https://tabs.ultimate-guitar.com/tab/10cc/im_n...,Im Not In Love,Ver 1,Im Not In Love 10cc,"[{'st': 0, 'et': 0.627, 'ochord': 'N'}, {'st':...","[N, Db:maj, A:maj, B:maj, C:maj, A:maj, B:maj,..."
16,16,16,46,5cada0079fc5af34a460ae35,10cc,"[F:maj, G:maj, F:maj, G:maj, A:maj, F:maj, D:m...",0,3.0,https://tabs.ultimate-guitar.com/tab/10cc/john...,Johnny Dont Do It,Ver 1,Johnny Dont Do It 10cc,"[{'st': 0, 'et': 2.345, 'ochord': 'A#'}, {'st'...","[Bb:maj, C:maj, D:maj, Bb:maj, G:maj, C:maj, B..."
17,17,17,47,5cada0079fc5af34a460ae3b,10cc,"[A:maj, E:maj, D:maj, A:maj, D:maj, A:maj, Db:...",0,9.0,https://tabs.ultimate-guitar.com/tab/10cc/old_...,Old Wild Men,Ver 1,Old Wild Men 10cc,"[{'st': 0, 'et': 1.881, 'ochord': 'N'}, {'st':...","[N, A:maj, E:maj, A:maj, D:maj, A:maj, Db:maj,..."
18,18,18,48,5cada0079fc5af34a460ae41,10cc,"[G:maj, A:maj, N, D:maj, D:maj, G:maj, D:maj, ...",0,21.0,https://tabs.ultimate-guitar.com/tab/10cc/rubb...,Rubber Bullets,Ver 1,Rubber Bullets 10cc,"[{'st': 0, 'et': 7.523, 'ochord': 'G'}, {'st':...","[G:maj, A:maj, C:maj, D:maj, E:min, G:maj, D:m..."
19,19,19,49,5cada0079fc5af34a460ae49,10cc,"[Ab:maj, Ab:maj, Ab:maj, Ab:maj, Ab:maj, Ab:ma...",0,9.0,https://tabs.ultimate-guitar.com/tab/10cc/some...,Somewhere In Hollywood,Ver 1,Somewhere In Hollywood 10cc,"[{'st': 0, 'et': 10.403, 'ochord': 'N'}, {'st'...","[N, Gb:maj, Eb:maj, Gb:maj, Ab:maj, Gb:maj, Ab..."


In [43]:
print(len(df2),'個Question(Song)')

9023 個Question(Song)


In [16]:
print(status(df2['song_chord'].str.len()),status(df2['chord'].str.len()))



總數        9023.000000
最小值         25.000000
最小值位置      797.000000
5%分位數       48.000000
25%分位數      73.000000
中位數         92.000000
75%分位數     112.000000
95%分位數     139.000000
均值          92.483653
最大值        158.000000
最大值位數     1541.000000
平均絕對偏差      22.215157
方差         744.402942
標準差         27.283749
偏度           0.075139
峰度          -0.481650
dtype: float64 總數        9023.000000
最小值         25.000000
最小值位置      744.000000
5%分位數       34.000000
25%分位數      57.000000
中位數         78.000000
75%分位數     102.000000
95%分位數     135.000000
均值          80.286601
最大值        158.000000
最大值位數     1701.000000
平均絕對偏差      25.205449
方差         933.600183
標準差         30.554872
偏度           0.316013
峰度          -0.610010
dtype: float64


# QA_train_df:QA模型訓練用的資料表 <br>


In [17]:
#QA_train_df, QA_test_df = train_test_split(df2, test_size=0.3)
QA_train_df=df2.copy()

In [18]:
print('QA_train_df長度:',len(QA_train_df))


QA_train_df長度: 9023


從QA_train_df製作三個資料: <br>
1.questions <br>
2.good_anwsers <br>
3.all_answers

In [19]:
chord_vector = {
               
        'C:maj':  [0,4,7],
        'Db:maj': [1,5,8],
        'Db:min': [1,4,8],
        'D:maj':  [2,7,9],
        'Eb:maj': [3,7,10],
        'Eb:min': [3,6,10],
        'E:maj':  [4,8,11],
        'E:min':  [4,7,11],
        'F:maj':  [0,5,9],
        'F:min':  [0,4,9],
        'Gb:maj': [1,6,10],
        'Gb:min': [1,6,9],
        'G:maj':  [2,7,11],
        'G:min':  [2,7,10],
        'Ab:maj': [0,3,8],
        'Ab:min': [3,8,11],
        'A:maj':  [1,4,9],
        'A:min':  [0,4,9],
        'Bb:maj': [2,5,10],
        'Bb:min': [1,5,10],
        'B:maj':  [2,6,11],
        'B:min':  [1,6,11]    
   
    }
chord_index={
    'C:maj':  1,
    'Db:maj': 2,
    'Db:min': 3,
    'D:maj':  4,
    'Eb:maj': 5,
    'Eb:min': 6,
    'E:maj':  7,
    'E:min':  8,
    'F:maj':  9,
    'F:min':  10,
    'Gb:maj': 11,
    'Gb:min': 12,
    'G:maj':  13,
    'G:min':  14,
    'Ab:maj': 15,
    'Ab:min': 16,
    'A:maj':  17,
    'A:min':  18,
    'Bb:maj': 19,
    'Bb:min': 20,
    'B:maj':  21,
    'B:min':  22    

}

In [22]:
num=0
questions=[]
for index, row in QA_train_df.iterrows():
    num +=1
    if (num%1000)==0:
        print(num)
        
    
    questions_row=np.array([])
    for chord in row['song_chord']:
        
        index=chord_index.get(chord)
        if index is None:index=np.zeros(1)
        if index is not None:index=np.array(index)
        questions_row=np.hstack([questions_row, index]) if questions_row.size else index
    questions.append(questions_row)
    
num=0
good_answers=[]
    
for index, row in QA_train_df.iterrows():
    num +=1
    if (num%1000)==0:
        print(num)


    answers_row=np.array([])
    row_chord=row['chord']
   
    if row['song_name'] in has_2ver[1000:]:
            df_tmp=df[df.song_name==row['song_name']]
            df_tmp=df_tmp[~(df_tmp.version==row['version'])]
            min_index=df_tmp.rating.argmin()
            row_chord=df_tmp.loc[min_index].chord
    
    for chord in row_chord:
        
        
        
            
        index=chord_index.get(chord)
        if index is None:index=np.zeros(1)
        if index is not None:index=np.array(index)
        answers_row=np.hstack([answers_row, index]) if answers_row.size else index
    good_answers.append(answers_row)



print('ok')

    

1000
2000
3000
4000
5000
6000
7000
8000
9000
1000
2000
3000
4000
5000
6000
7000
8000
9000
ok


從df裡拿到所有的吉他譜和弦當成All_answers <br>

之後要從All_answers裡抽樣bad answers

In [21]:
all_answers=[]
num=0
for row in df.chord:
    num +=1
    if (num%1000)==0:
        print(num)


    answers_row=np.array([])
    for chord in row:

        index=chord_index.get(chord)
        if index is None:index=np.zeros(1)
        if index is not None:index=np.array(index)
        answers_row=np.hstack([answers_row, index]) if answers_row.size else index
    all_answers.append(answers_row)
print('ok')

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
ok


In [23]:
def find_max_length_of_songs(Q,A):
    max_length=0
    for i in Q:
        if max_length<len(i):max_length=len(i)
    for i in A:
        if max_length<len(i):max_length=len(i)
    return max_length

要知道最多和弦的歌曲，和弦數量是多少，然後把每首歌的和弦數目pad成一樣長

In [24]:

find_max_length_of_songs(questions,good_answers)

158

In [25]:
#pad成158個row(2d matrix,149X3)
def pad_QA(QA):
    data=QA[:]
    
    for i in range(len(data)):
        if (i%1000)==0:print(i)
        
        a=data[i]
        padding_length=158-len(a)
      
        data[i]=np.pad(a, [(0, padding_length)], mode='constant')
      
           
    return data

In [26]:
questions=pad_QA(questions)
good_answers=pad_QA(good_answers)
print('ok')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
ok


In [27]:
all_answers=pad_QA(all_answers)
print('ok')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
ok


In [29]:
#檢查padding後的資料長度
len(all_answers[17000])

158

In [30]:
import pickle

out = open("./QA_data/0511/questions0501","wb")
pickle.dump(questions, out)
out = open("./QA_data/0511/good_answers0501","wb")
pickle.dump(good_answers, out)
del out
gc.collect()
print('ok')

ok


In [31]:
import pickle

out = open("./QA_data/0511/all_answers0501","wb")
pickle.dump(all_answers, out)
print('ok')

ok
